## Notebook Preface

## Constructs the working folder

* Positions the project folder in the Google Drive.
  1. From "Share with me", right click on "W266 Final Project", select "Add shortcut to Drive"
  2. "W266 Final Project" will show up in "MyDrive"

* Mounts the Google Drive at /content/drive in the Colab runtime.

* Defines the working folder relative to /content/drive.



In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
working_folder = "/content/drive/MyDrive/W266 Final Project/CnF/PhotoStoryGenerator"
testing_json = f"{working_folder}/test_hints.json"


## Imports libraries

In [3]:
# First upload the training and evaluation files to this runtime (Press connect if needed)
!pip install transformers torch


     |████████████████████████████████| 2.6 MB 4.3 MB/s 
     |████████████████████████████████| 636 kB 58.2 MB/s 
     |████████████████████████████████| 3.3 MB 61.2 MB/s 
     |████████████████████████████████| 895 kB 57.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import json
import logging
import math
import os
import re
from dataclasses import dataclass, field
from typing import Optional

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from transformers import (
    AutoConfig,
    GPT2LMHeadModel,
    AutoTokenizer,
    TextGenerationPipeline,
    pipeline,
    set_seed,
)

# Setup logging
logger = logging.getLogger(__name__)

from IPython.display import HTML, display
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)


In [5]:
#METEOR
!pip install nltk==3.5
import nltk
nltk.download('wordnet')

#BLEU
from nltk.translate.bleu_score import sentence_bleu

#ROUGE
!pip install rouge-score
from rouge_score import rouge_scorer
Rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

#BERT SCORE
!pip install bert_score
from bert_score import BERTScorer
BERT_scorer = BERTScorer(lang="en", rescale_with_baseline=True)


     |████████████████████████████████| 1.4 MB 1.8 MB/s 
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434689 sha256=b9f2b9f38c77ed2cbc48b202c085792b4efc1647ad2786d403df1404648a1d4f
  Stored in directory: /root/.cache/pip/wheels/45/6c/46/a1865e7ba706b3817f5d1b2ff7ce8996aabdd0d03d47ba0266
Successfully built nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


     |████████████████████████████████| 59 kB 2.8 MB/s 


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Defines score computer

In [6]:
def get_scores(reference, hypothesis):
    rouge = Rouge_scorer.score(reference, hypothesis)
    P, R, F1 = BERT_scorer.score([hypothesis], [reference])

    return {
        "meteor": nltk.translate.meteor_score.meteor_score([reference], hypothesis),
        "rouge1": { "precision": rouge["rouge1"].precision, "recall": rouge["rouge1"].recall, "fmeasure": rouge["rouge1"].fmeasure },
        "rougeL": { "precision": rouge["rougeL"].precision, "recall": rouge["rougeL"].recall, "fmeasure": rouge["rougeL"].fmeasure },
        "bleu": sentence_bleu([reference.split()], hypothesis.split(), weights=(1, 0, 0, 0)),
        "bert_score": { "precision": P.item(), "recall": R.item(), "f1": F1.item() }
    }


## Constructs the testing dataset

In [7]:
def load_dataset_for_testing(num_records):
    results = []
    with open (testing_json) as f:
        _json = json.load(f)
        print(f"Generate {num_records} of {len(_json.keys())} records")
        for s in _json:
           story = _json[s]
           hints = story["hints"]
           results.append({
               "story_id": s,
               "reference": story["sis"],
               "prefix": " ".join(hints[1]),
               "hints": ' '.join(hints[1]),
               "urls": story["urls"]
           })
           num_records -= 1
           if num_records <= 0:
              break
        return results


## Creates the story generation pipeline

In [8]:
@dataclass
class TextGenerationArguments:
    _num_tests: int = 100
    _output: str = None
    _parsing_regex: re = None
    max_length: int = 500
    num_return_sequences: Optional[int] = None
    early_stopping: bool = False
    return_full_text: bool = False

    # Sampling
    do_sample: Optional[bool] = None

    # Redistribute the probability for the top K words
    # Limit the sampling pool
    # Lmiting the sample pool to a fixed size K could endanger the model to
    #   produce gibberish for sharp distributions and
    #   limit the model's creativity for flat distribution.
    # Default: 50
    top_k: Optional[int] = None

    # Dynamically set the size of the sampling pool
    #   with the probability of the selected words summed up to p%
    # Default: 1.0
    top_p: Optional[float] = None

    # The lower the temperature
    # 1. The more deterministic the output
    # 2. Word of higher probability is chosen
    # 3. Lower temperature to fix gibberish
    # 4. 0: most deterministic, probably more repetition
    # Default: 1.0
    temperature: Optional[float] = None

    # 1: No penalty
    # Infinity: Max penalty
    repetition_penalty: Optional[float] = None

    # Beam Search
    num_beams: Optional[int] = None
    no_repeat_ngram_size: Optional[int] = None


@dataclass
class ModelArguments:
    model_name_or_path: Optional[str] = None
    cache_dir: Optional[str] = None
    model_type: Optional[str] = None

model_args = ModelArguments(model_name_or_path="gpt2-medium", model_type="gpt2-medium")
config = AutoConfig.from_pretrained(model_args.model_name_or_path, cache_dir=model_args.cache_dir)

model = GPT2LMHeadModel.from_pretrained(
    model_args.model_name_or_path,
    from_tf=bool(".ckpt" in model_args.model_name_or_path),
    config=config,
    cache_dir=model_args.cache_dir,
)

tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path, cache_dir=model_args.cache_dir)
story_generator = TextGenerationPipeline(model=model, tokenizer=tokenizer, device=0)


In [9]:
def dump_data(_data, _file):
    with open(f"{_file}", "w") as outfile:
        json.dump(_data, outfile)


## Text generation

In [10]:
story_regex = re.compile("^(?P<SENT>[^.!?]+[.!?][^.!?]+[.!?][^.!?]+[.!?][^.!?]+[.!?][^.!?]+[.!?]).+", re.MULTILINE | re.IGNORECASE)

configs = [
    TextGenerationArguments(
        _num_tests = 3385,
        _output = "BL_BS_B5_TestSet",
        _parsing_regex = story_regex,
        max_length = 500,
        num_beams = 5,
        no_repeat_ngram_size = 3,
        num_return_sequences = 1,
        repetition_penalty = 1.1,
    ),
]

In [11]:
for config in configs:
    data = load_dataset_for_testing(config._num_tests)
    print("Configuration:", config._output)

    set_seed(0)
    for i, d in enumerate(data):
        stories = story_generator(d["prefix"], **(config.__dict__))

        best = {"text": "", "scores": {
            "meteor": -100,
            "rouge1": { "precision": -100, "recall": -100, "fmeasure": -100 },
            "rougeL": { "precision": -100, "recall": -100, "fmeasure": -100 },
            "bleu": -100,
            "bert_score": { "precision": -100, "recall": -100, "f1": -100 },
        }}

        for story in stories:
            gen_text = story["generated_text"].replace("\n", "").strip()
            match = story_regex.match(gen_text)
            if match is not None:
                hypothesis = match.group("SENT")
            else:
                hypothesis = " ".join(gen_text.split()[:75])
            if len(hypothesis) == 0:
                continue

            scores = get_scores(d["hints"], hypothesis)
            if scores["meteor"] > best["scores"]["meteor"]:
                best = { "text": hypothesis, "scores": scores }

        best["scores"] = get_scores(d["reference"], best["text"])
        best["dii_scores"] = get_scores(d["reference"], d["hints"])
        d[config._output] = best
        print(f"{i+1} of {config._num_tests}: METEOR(text)={best['scores']['meteor']}, METOR(dii)={best['dii_scores']['meteor']}")

    dump_data(data, f"{working_folder}/{config._output}_results.json")
    print("Save results:", f"{config._output}_results.json")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generate 3385 of 3385 records
Configuration: BL_BS_B5_TestSet


/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts o

1 of 3385: METEOR(text)=0.10174418604651164, METOR(dii)=0.12283236994219653


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

3382 of 3385: METEOR(text)=0.16, METOR(dii)=0.13


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


3383 of 3385: METEOR(text)=0.12801204819277112, METOR(dii)=0.1328125


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


3384 of 3385: METEOR(text)=0.12631578947368421, METOR(dii)=0.1973885193397388
3385 of 3385: METEOR(text)=0.14940239043824702, METOR(dii)=0.18623895862389586
Save results: BL_BS_B5_TestSet_results.json
